In [34]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import splitfolders

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Tuple
import optuna
from optuna.integration import TFKerasPruningCallback

Prepare training data
===

In [2]:
train_src = "asl_alphabet_train"
train_dir = 'datasets/train'
val_dir = 'datasets/val'
test_dir  = 'datasets/test'
splitfolders.ratio(train_src, output="datasets",
    seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False) # 產生trian(訓練集)、val(驗證集)、test(測試集)

Copying files: 87000 files [00:47, 1818.73 files/s]


Preprocessing and Get labels
===

In [11]:
@dataclass(frozen=True)
class dataconfig:
    batch_size: int = 32
    target_size: Tuple[int, int] = (32,32)

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
val_datagen   = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_datagen  = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory( # 多目錄時，目錄名為標籤
        train_dir,
        target_size=dataconfig.target_size,
        batch_size=dataconfig.batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=True)

val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=dataconfig.target_size,
        batch_size=dataconfig.batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=False)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=dataconfig.target_size,
        batch_size=dataconfig.batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=False)

Found 69600 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [13]:
labels = list(train_generator.class_indices.keys())
print(labels)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


Model
===

In [24]:
@dataclass(frozen=True)
class modelconfig:
    num_classes: int = len(labels)
    input_shape: Tuple[int,int,int] = (32,32,1)
    dropout: float = 0.1

@dataclass(frozen=True)
class traingingconfig:
    checkpoint_path: str = "model/B_best_model.keras"
    epochs: int = 10

In [35]:
def objective(trial):
    model_config = modelconfig(
        input_shape=(32, 32, 1),  # Adjust for MNIST
        dropout=trial.suggest_float("dropout", 0.1, 0.5)
    )
    model = models.Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=model_config.input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(model_config.dropout))
    
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(model_config.dropout))
    
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(model_config.dropout))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(Dropout(model_config.dropout*2))
    model.add(layers.Dense(model_config.num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = model.fit(train_generator,
                        validation_data=val_generator,
                        epochs=traingingconfig.epochs,
                        callbacks=[early_stopping, TFKerasPruningCallback(trial, 'val_loss')],
                        verbose=1)
    val_loss = min(history.history['val_loss'])
    return val_loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2024-05-18 11:41:55,854] A new study created in memory with name: no-name-05d4f980-d39f-4266-927c-4d5564aa105e


Epoch 1/10
2175/2175 [==============================] - 148s 67ms/step - loss: 1.6373 - accuracy: 0.4860 - val_loss: 0.8197 - val_accuracy: 0.7323
Epoch 2/10
2175/2175 [==============================] - 95s 44ms/step - loss: 0.6350 - accuracy: 0.7812 - val_loss: 0.6128 - val_accuracy: 0.7946
Epoch 3/10
2175/2175 [==============================] - 104s 48ms/step - loss: 0.4145 - accuracy: 0.8567 - val_loss: 0.5720 - val_accuracy: 0.8201
Epoch 4/10
1217/2175 [===============>..............] - ETA: 1:06 - loss: 0.3327 - accuracy: 0.8854

In [34]:
best_trial = study.best_trial
print(f"Best trial parameters: {best_trial.params}")
print(f"Best trial value: {best_trial.value}")

272/272 [==============================] - 11s 42ms/step - loss: 0.0891 - accuracy: 0.9708
Test loss:  0.08905266225337982
Test accuracy:  0.9708045721054077
